# Read hourly data, write daily data

In [ ]:
import xarray as xr
import dask.distributed
from dask.distributed import Client
import fsspec

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
import ebdpy as ebd

ebd.set_credentials(profile='esip-qhub')

profile = 'esip-qhub'
region = 'us-west-2'
endpoint = f's3.{region}.amazonaws.com'
ebd.set_credentials(profile=profile, region=region, endpoint=endpoint)
worker_max = 10
client,cluster = ebd.start_dask_cluster(profile=profile,worker_max=worker_max, 
                                      region=region, use_existing_cluster=True,
                                      adaptive_scaling=False, wait_for_cluster=False, 
                                      environment='pangeo', worker_profile='Pangeo Worker', propagate_env=True)

In [ ]:
client

In [ ]:
fs1 = fsspec.filesystem('s3', anon=True)
read_url = 's3://noaa-nwm-retro-v2-zarr-pds'
read_mapper = fs1.get_mapper(read_url)

In [ ]:
fs2 = fsspec.filesystem('s3', anon=False)
write_url = 's3://esip-qhub/usgs/rsignell/zarr/zarr_daily'
write_mapper = fs2.get_mapper(write_url)

In [ ]:
%%time
ds = xr.open_zarr(read_mapper, consolidated=True)

In [ ]:
ds.streamflow

In [ ]:
ds2 = ds.sel(time='2017-01')

In [ ]:
#client.scatter(ds2)

In [ ]:
daily_mean = ds2.resample(time='D').mean()

In [ ]:
daily_mean.streamflow

In [ ]:
daily_mean.attrs = ds.attrs

In [ ]:
daily_mean = daily_mean.chunk({'time':672})

In [ ]:
daily_mean.streamflow

In [ ]:
def delete_s3(url):
    fs1 = fsspec.open(url, anon=False).fs
    if fs1.exists(url):
        fs1.rm(url, recursive=True)

In [ ]:
delete_s3(write_url)

In [ ]:
write_url

In [ ]:
%%time
a = daily_mean.to_zarr(write_mapper, compute=False, mode='w', consolidated=True)

In [ ]:
len(a.dask)

In [ ]:
fs2.ls(write_url)

In [ ]:
%%time
dask.compute(a, retries=10)

In [ ]:
26*12*2.5/60/4

In [ ]:
def _apply_along_axis(arr, func1d, func1d_axis):
    """Wrap apply_along_axis"""
    return np.apply_along_axis(func1d, func1d_axis, arr)


def apply_along_axis(func1d, axis, arr):
    """Apply the harmonic analysis to 1-D slices along the given axis."""
    arr = dask.array.core.asarray(arr)

    # Validate and normalize axis.
    arr.shape[axis]
    axis = len(arr.shape[:axis])

    # Test out some data with the function.
    test_data = np.ones(args[0].shape[1], dtype=arr.dtype)
    test_result = np.array(func1d(test_data))

    # Map analyze over the data to get the result
    # Adds other axes as needed.
    result = arr.map_blocks(
        _apply_along_axis,
        name=dask.utils.funcname(func1d) + '-along-axis',
        dtype=test_result.dtype,
        chunks=(arr.chunks[:axis] + test_result.shape + arr.chunks[axis + 1:]),
        drop_axis=axis,
        new_axis=list(range(axis, axis + test_result.ndim, 1)),
        func1d=func1d,
        func1d_axis=axis
    )

    return result

In [ ]:
import numpy as np

In [ ]:
future = apply_along_axis(np.mean, 0, da)